<a href="https://colab.research.google.com/github/amalsalilan/IPL_Infographics_Data_Analytics_-_Data_Visualization_Infosys_Internship_Oct2024/blob/K_Akshya/08_Model_Evaluation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

In [3]:
df=pd.read_csv('/content/all_season_details_data.csv')
df.head()

<ipython-input-3-20ecdbe3bc5d>:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/all_season_details_data.csv')


,season,match_id,match_name,home_team,away_team,current_innings,innings_id,over,ball,runs,...,bowler2_maidens,bowler2_runs,bowler2_wkts,wicket_id,wkt_batsman_name,wkt_bowler_name,wkt_batsman_runs,wkt_batsman_balls,wkt_text,isRetiredHurt
0,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [4]:
new_df = df[['season', 'match_id', 'home_team', 'away_team', 'current_innings',
                      'innings_id', 'over', 'ball', 'runs', 'wicket_id','bowler1_name',
                      'wkt_batsman_name', 'wkt_batsman_runs']].copy()
# Calculate batsman's strike rate in each match/innings
new_df['strike_rate'] = (new_df['runs'] / new_df['ball']) * 100
new_df['strike_rate'] = new_df['strike_rate'].fillna(0)

new_df['over_runs'] = new_df.groupby(['match_id', 'current_innings', 'over'])['runs'].transform('sum')
new_df['over_run_rate'] = new_df['over_runs'] / (new_df['over'] + 1.0)

new_df['cumulative_runs'] = new_df.groupby(['match_id', 'current_innings'])['runs'].cumsum()

new_df['run_rate'] = new_df['cumulative_runs'] / (new_df['over'] + 0.1)

# Assuming 'wkt_batsman_name' indicates a wicket


new_df['wickets_lost'] = new_df.groupby(['match_id', 'current_innings'])['wkt_batsman_runs']\
                                                 .transform(lambda x: (x >= 0).cumsum())

new_df['cumulative_balls'] = new_df.groupby(['match_id', 'current_innings']).cumcount() + 1
new_df['cumulative_strike_rate'] = (new_df['cumulative_runs'] / new_df['cumulative_balls']) * 100


In [5]:
new_df['wickets_taken'] = new_df['wkt_batsman_name'].notnull().astype(int)

# Calculate cumulative wickets for each bowler in the given match and innings
new_df['bowler_wickets'] = new_df.groupby(['match_id', 'current_innings', 'bowler1_name'])['wickets_taken'].cumsum()
result_df = new_df[['match_id', 'current_innings', 'bowler1_name', 'bowler_wickets']]

In [6]:
unique_home_teams = new_df['home_team'].unique().tolist()
home_teams_df = pd.DataFrame(unique_home_teams, columns=['Teams'])
print(home_teams_df)

    Teams
0      GT
1    PBKS
2     LSG
3     SRH
4     RCB
5     CSK
6      DC
7      RR
8     KKR
9      MI
10   KXIP
11    RPS
12     GL
13    PWI
14  Kochi


In [7]:
team_mapping = {'GT': 0, 'PBKS': 1, 'LSG': 2, 'SRH': 3,'RCB': 4, 'CSK': 5, 'DC': 6, 'RR': 7,
                'KKR': 8, 'MI': 9, 'KXIP': 10, 'RPS': 11,'GL': 12, 'PWI': 13, 'KOCHI': 14}
new_df['home_team_encoded'] = new_df['home_team'].map(team_mapping)
new_df['away_team_encoded'] = new_df['away_team'].map(team_mapping)
new_df['current_innings'] = new_df['current_innings'].map(team_mapping)
new_df.fillna(0, inplace=True)
df_result=new_df[['home_team_encoded','away_team_encoded','current_innings']]
df_result

,home_team_encoded,away_team_encoded,current_innings
0,0.0,5.0,5.0
1,0.0,5.0,5.0
2,0.0,5.0,5.0
3,0.0,5.0,5.0
4,0.0,5.0,5.0
...,...,...,...
242544,5.0,7.0,7.0
242545,5.0,7.0,7.0
242546,5.0,7.0,7.0
242547,5.0,7.0,7.0


In [8]:
X = new_df[['home_team_encoded','away_team_encoded','current_innings','over', 'ball', 'run_rate','wickets_lost','wickets_taken','cumulative_strike_rate']]
y = new_df['cumulative_runs']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
X_train

,home_team_encoded,away_team_encoded,current_innings,over,ball,run_rate,wickets_lost,wickets_taken,cumulative_strike_rate
199032,13.0,8.0,13.0,12,4,5.537190,3,0,93.055556
237646,6.0,3.0,6.0,17,1,8.596491,2,0,144.117647
10462,0.0,6.0,6.0,1,5,4.545455,1,0,100.000000
39475,7.0,4.0,7.0,8,4,8.518519,0,0,150.000000
137928,3.0,5.0,3.0,9,2,6.153846,2,0,112.000000
...,...,...,...,...,...,...,...,...,...
119879,8.0,3.0,3.0,10,6,7.029703,3,0,116.393443
103694,3.0,9.0,3.0,12,4,8.016529,1,0,138.571429
131932,10.0,4.0,4.0,6,1,8.196721,2,0,151.515152
146867,8.0,3.0,3.0,18,6,7.182320,5,0,117.117117


In [11]:
X_test

,home_team_encoded,away_team_encoded,current_innings,over,ball,run_rate,wickets_lost,wickets_taken,cumulative_strike_rate
122566,7.0,9.0,9.0,10,5,4.455446,4,1,76.271186
147656,9.0,5.0,9.0,5,2,6.078431,0,0,119.230769
27099,0.0,3.0,0.0,1,1,0.000000,0,0,0.000000
42493,5.0,9.0,5.0,2,5,0.952381,2,0,18.181818
154359,3.0,10.0,3.0,19,1,5.863874,5,0,102.752294
...,...,...,...,...,...,...,...,...,...
19877,0.0,6.0,0.0,17,5,8.362573,3,0,138.834951
217199,5.0,6.0,5.0,14,3,9.503546,3,0,157.647059
188029,8.0,7.0,7.0,3,1,4.516129,0,0,107.692308
220380,8.0,9.0,8.0,15,1,6.092715,2,0,102.222222


In [12]:
y_train

,cumulative_runs
199032,67
237646,147
10462,5
39475,69
137928,56
...,...
119879,71
103694,97
131932,50
146867,130


In [13]:
y_test

,cumulative_runs
122566,45
147656,31
27099,0
42493,2
154359,112
...,...
19877,143
217199,134
188029,14
220380,92


**Train the Random Forest model**

In [14]:
rf_model=RandomForestRegressor()

In [15]:
rf_model.fit(X_train,y_train)

RandomForestRegressor()

In [16]:
y_rf_simple_pred = rf_model.predict(X_test)

In [17]:
rf_simple_mse=mean_squared_error(y_test,y_rf_simple_pred)
rf_r2=r2_score(y_test,y_rf_simple_pred)

In [18]:
print('MSE score: ',rf_simple_mse)

MSE score:  0.07991976705833842


In [19]:
print('R2 score: ',rf_r2)

R2 score:  0.9999679773212595


**Linear Regression**

In [20]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

In [21]:
pred = lr_model.predict(X_test)
lr_mse=mean_squared_error(y_test,pred)

In [22]:
print('MSE score: ',lr_mse)
lr_r2=r2_score(y_test,pred)
print('R2 score: ',lr_r2)

MSE score:  234.00465804614657
R2 score:  0.906237764895929


**Polynomial Regression**

In [23]:
poly = PolynomialFeatures(degree=2)
X_polya = poly.fit_transform(X_train)
X_polyb=poly.fit_transform(X_test)

In [24]:
poly_model = LinearRegression()
poly_model.fit(X_polya, y_train)
pred = poly_model.predict(X_polyb)

In [25]:
pr_mse=mean_squared_error(y_test,pred)
print('MSE score: ',pr_mse)

pr_r2=r2_score(y_test,pred)
print('R2 score: ',pr_r2)

MSE score:  4.608998684827578e-24
R2 score:  1.0


**XGB Regressor**

In [26]:
xgb_model = XGBRegressor(random_state=42)
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [27]:
y_pred_xgb = xgb_model.predict(X_test)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

In [28]:
print("XGBoost Regressor:")
print("MSE score: ",mse_xgb)
print("R2 score: ",r2_xgb)

XGBoost Regressor:
MSE score:  0.3585136782550974
R2 score:  0.9998563528060913


**Decison tree Regression**

In [29]:
tree_model = DecisionTreeRegressor(max_depth=3)
tree_model.fit(X_train, y_train)
pred = tree_model.predict(X_test)
tree_mse=mean_squared_error(y_test,pred)
print('MSE score: ',tree_mse)
tree_r2=r2_score(y_test,pred)
print('R2 score: ',tree_r2)

MSE score:  327.7749015281932
R2 score:  0.8686654033517592


**Gradient BoostRegression**

In [30]:
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train, y_train)

GradientBoostingRegressor(random_state=42)

In [31]:
y_pred_gb = gb_model.predict(X_test)

In [32]:
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

In [33]:
print("Gradient Boosting Regressor:")
print("MSE score: ",mse_gb)
print("R2 score: ",r2_gb)

Gradient Boosting Regressor:
MSE score:  3.084040290028468
R2 score:  0.9987642702792393


In [34]:
model_comparison = pd.DataFrame({
    'Model': ['Random Forest', 'Linear Regression', 'Polynomial Regression','Gradient Boosting','XGBoost Regressor'],
    'MSE': [rf_simple_mse, lr_mse, pr_mse,mse_gb,mse_xgb],
    'R² Score': [rf_r2, lr_r2, pr_r2,r2_gb,r2_xgb]
})
print(model_comparison)

                   Model           MSE  R² Score
0          Random Forest  7.991977e-02  0.999968
1      Linear Regression  2.340047e+02  0.906238
2  Polynomial Regression  4.608999e-24  1.000000
3      Gradient Boosting  3.084040e+00  0.998764
4      XGBoost Regressor  3.585137e-01  0.999856


**HyperParameter Tuning**

---



In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

**Making a Predictions**

---



In [40]:
new_data = pd.DataFrame({
    'home_team_encoded': [13],
    'away_team_encoded': [8],
    'current_innings':[13],
    'over': [12],
    'ball': [4],
    'run_rate': [5.537],
    'wickets_lost': [3],
    'wickets_taken':[0],
    'cumulative_strike_rate':[93]
})

In [41]:
predict_runs=rf_model.predict(new_data)       #Prediction by random forest
print(f"Predicted Cumulative Runs: {predict_runs[0]}")

Predicted Cumulative Runs: 67.0


In [42]:
predicted_runs_lr = lr_model.predict(new_data)                   #prediction by linear regression
print(f"Predicted Cumulative Runs: {predicted_runs_lr[0]}")

Predicted Cumulative Runs: 79.86925054280805


In [43]:
predicted_runs_rdg = gb_model.predict(new_data)                 #prediction by gradient boost regression
print(f"Predicted Cumulative Runs: {predicted_runs_rdg[0]}")

Predicted Cumulative Runs: 68.06745705428104


In [44]:
predicted_runs_xgb = xgb_model.predict(new_data)                  #prediction by xgboost
print(f"Predicted Cumulative Runs: {predicted_runs_xgb[0]}")

Predicted Cumulative Runs: 65.25233459472656


In [45]:
predicted_runs_tree = tree_model.predict(new_data)                  #prediction by decision tree
print(f"Predicted Cumulative Runs: {predicted_runs_tree[0]}")

Predicted Cumulative Runs: 82.07005637545333


**Report**

The correct decsion is given by random forest model


In [46]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(
    random_state=42
)

In [47]:
param_grid = {
    'n_estimators': [50, 100, 150],  # Reduce the range or number of values
    'max_depth': [10, 20],           # Focus on reasonable ranges
    'min_samples_split': [2, 5]      # Fewer values
}

In [48]:
grid_search=GridSearchCV(estimator=rf_model,
                         param_grid=param_grid,
                        scoring='r2',
                        cv=3,
                        verbose=3,
                        refit=True,
                        return_train_score=True)

In [49]:
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3] END max_depth=10, min_samples_split=2, n_estimators=50;, score=(train=1.000, test=1.000) total time=  15.6s
[CV 2/3] END max_depth=10, min_samples_split=2, n_estimators=50;, score=(train=1.000, test=1.000) total time=  15.5s
[CV 3/3] END max_depth=10, min_samples_split=2, n_estimators=50;, score=(train=1.000, test=1.000) total time=  16.1s
[CV 1/3] END max_depth=10, min_samples_split=2, n_estimators=100;, score=(train=1.000, test=1.000) total time=  30.2s
[CV 2/3] END max_depth=10, min_samples_split=2, n_estimators=100;, score=(train=1.000, test=1.000) total time=  29.9s
[CV 3/3] END max_depth=10, min_samples_split=2, n_estimators=100;, score=(train=1.000, test=1.000) total time=  30.9s
[CV 1/3] END max_depth=10, min_samples_split=2, n_estimators=150;, score=(train=1.000, test=1.000) total time=  45.2s
[CV 2/3] END max_depth=10, min_samples_split=2, n_estimators=150;, score=(train=1.000, test=1.000) total time=  44.7

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42),
             param_grid={'max_depth': [10, 20], 'min_samples_split': [2, 5],
                         'n_estimators': [50, 100, 150]},
             return_train_score=True, scoring='r2', verbose=3)

In [50]:
best_parameter=grid_search.best_params_
best_score=grid_search.best_score_
best_model=grid_search.best_estimator_

In [51]:
print("Best Parameters:", best_parameter)
print("Best R² Score:", best_score)

Best Parameters: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 100}
Best R² Score: 0.999949840417291
